In [9]:
import pandas as pd
import numpy as np
import torch
import tqdm
import os
import random
from torch.utils.data import Dataset, DataLoader
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="4"
tf_device='/gpu:0'

In [3]:
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
config = BertConfig.from_pretrained("indobenchmark/indobert-base-p1")
config.num_labels = 3
model = BertForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p1", config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
class DocumentSentimentDataset(Dataset):
    # Static constant variable (We need to have this part to comply with IndoNLU standard)
    LABEL2INDEX = {'negative': 0, 'neutral': 1, 'positive': 2} # Label string to index
    INDEX2LABEL = {0:'negative', 1:'neutral', 2: 'positive'} # Index to label string
    NUM_LABELS = 3 # Number of label
   
    def load_dataset(self, path):
        df = pd.read_csv(path, sep='\t', header=None) # Read tsv file with pandas
        df.columns = ['text','sentiment'] # Rename the columns
        df['sentiment'] = df['sentiment'].apply(lambda lab: self.LABEL2INDEX[lab]) # Convert string label into index
        return df
   
    def __init__(self, dataset_path, tokenizer, *args, **kwargs):
        self.data = self.load_dataset(dataset_path) # Load the tsv file

        # Assign the tokenizer for tokenization
        # here we use subword tokenizer from HuggingFace
        self.tokenizer = tokenizer
    
    def __getitem__(self, index):
        data = self.data.loc[index,:] # Taking data from a specific row from Pandas
        text, sentiment = data['text'], data['sentiment'] # Take text and sentiment from the row
        subwords = self.tokenizer.encode(text) # Tokenize the text with tokenizer
        return np.array(subwords), np.array(sentiment), data['text']
   
    def __len__(self):
        return len(self.data)  # Return the length of the dataset

In [5]:
class DocumentSentimentDataLoader(DataLoader):
    def __init__(self, max_seq_len=512, *args, **kwargs):
        super(DocumentSentimentDataLoader, self).__init__(*args, **kwargs)
        self.max_seq_len = max_seq_len # Assign max limit of the sequence length
        self.collate_fn = self._collate_fn # Assign the collate_fn function with our function
       
    def _collate_fn(self, batch):
        batch_size = len(batch) # Take the batch size
        max_seq_len = max(map(lambda x: len(x[0]), batch)) # Find maximum sequence length from the batch 
        max_seq_len = min(self.max_seq_len, max_seq_len) # Compare with our defined limit
       
    # Create buffer for subword, mask, and sentiment labels, initialize all with 0
        subword_batch = np.zeros((batch_size, max_seq_len), dtype=np.int64)
        mask_batch = np.zeros((batch_size, max_seq_len), dtype=np.float32)
        sentiment_batch = np.zeros((batch_size, 1), dtype=np.int64)
       
    # Fill all of the buffer
        seq_list = []
        for i, (subwords, sentiment, raw_seq) in enumerate(batch):
            subwords = subwords[:max_seq_len]
            subword_batch[i,:len(subwords)] = subwords
            mask_batch[i,:len(subwords)] = 1
            sentiment_batch[i,0] = sentiment
            
            seq_list.append(raw_seq)
           
    # Return the subword, mask, and sentiment data
        return subword_batch, mask_batch, sentiment_batch, seq_list

In [6]:
def forward_sequence_classification(model, batch_data, i2w, is_test=False, device = 'cuda', **kwargs):
    # Unpack batch data
    if len(batch_data) == 3:
        (subword_batch, mask_batch, label_batch) = batch_data
        token_type_batch = None
    elif len(batch_data) == 4:
        (subword_batch, mask_batch, token_type_batch, label_batch) = batch_data
    
    # Prepare input & label
    subword_batch = torch.LongTensor(subword_batch)
    mask_batch = torch.FloatTensor(mask_batch)
    token_type_batch = torch.LongTensor(token_type_batch) if token_type_batch is not None else None
    label_batch = torch.LongTensor(label_batch)
            
    if device == "cuda":
        subword_batch = subword_batch.cuda()
        mask_batch = mask_batch.cuda()
        token_type_batch = token_type_batch.cuda() if token_type_batch is not None else None
        label_batch = label_batch.cuda()

    # Forward model
    outputs = model(subword_batch, attention_mask=mask_batch, token_type_ids=token_type_batch, labels=label_batch)
    loss, logits = outputs[:2]
    
    # generate prediction & label list
    list_hyp = []
    list_label = []
    hyp = torch.topk(logits, 1)[1]
    for j in range(len(hyp)):
        list_hyp.append(i2w[hyp[j].item()])
        list_label.append(i2w[label_batch[j][0].item()])
    return loss, list_hyp, list_label

In [7]:
def document_sentiment_metrics_fn(list_hyp, list_label):
    metrics = {}
    metrics["ACC"] = accuracy_score(list_label, list_hyp)
    metrics["F1"] = f1_score(list_label, list_hyp, average='macro')
    metrics["REC"] = recall_score(list_label, list_hyp, average='macro')
    metrics["PRE"] = precision_score(list_label, list_hyp, average='macro')
    return metrics

In [10]:
import itertools
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed(241021)

In [11]:
w2i, i2w = DocumentSentimentDataset.LABEL2INDEX, DocumentSentimentDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'negative': 0, 'neutral': 1, 'positive': 2}
{0: 'negative', 1: 'neutral', 2: 'positive'}


In [13]:
text =  'satu satu aku sayang ibu, dua dua juga sayang ayah, tiga-tiga sayang adik kakak, satu dua tiga sayang semuanya'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
print(subwords)
logits = model(subwords)[0]
print(logits)
print(torch.topk(logits, k=1))
label = torch.topk(logits, k=1)[1].squeeze().item()
print(f'Text: {text} | Label : {i2w[label]} ({torch.nn.functional.softmax(logits, dim=-1).squeeze()[label] * 100}%)')

tensor([[    2,   282,   282,   304,  3067,  1066, 30468,   662,   662,   186,
          3067,  2161, 30468,  1224, 30469,  1224,  3067,  4265,  5213, 30468,
           282,   662,  1224,  3067,  2346,     3]])
tensor([[ 0.6718, -0.9024, -0.0760]], grad_fn=<AddmmBackward0>)
torch.return_types.topk(
values=tensor([[0.6718]], grad_fn=<TopkBackward0>),
indices=tensor([[0]]))
Text: satu satu aku sayang ibu, dua dua juga sayang ayah, tiga-tiga sayang adik kakak, satu dua tiga sayang semuanya | Label : negative (59.50214767456055%)


In [14]:
print(f'Text: {text} | Label : {i2w[label]} ({torch.nn.functional.softmax(logits, dim=-1).squeeze()[label] * 100}%)')

Text: satu satu aku sayang ibu, dua dua juga sayang ayah, tiga-tiga sayang adik kakak, satu dua tiga sayang semuanya | Label : negative (59.50214767456055%)


In [15]:
train = DocumentSentimentDataset('dataset/train_preprocess.tsv', tokenizer)
valid = DocumentSentimentDataset('dataset/valid_preprocess.tsv', tokenizer)
train_loader = DocumentSentimentDataLoader(dataset=train, max_seq_len=100, batch_size=32, num_workers=1, shuffle=True)
valid_loader = DocumentSentimentDataLoader(dataset=valid, max_seq_len=100, batch_size=32, num_workers=1, shuffle=True)

In [16]:
torch.cuda.empty_cache()

In [17]:
def count_param(module, trainable=True):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
count_param(model)

124443651

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-6)
model = model.cuda()

In [20]:
n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)
 
    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm.tqdm(iter(train_loader), leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        print((batch_data[:-1]))
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

(Epoch 1) TRAIN LOSS:0.5542 LR:0.00000300:   4%|█▋                                       | 1/25 [00:00<00:04,  4.94it/s]

(array([[    2,  1753,  8302, ...,     0,     0,     0],
       [    2,    34,  2999, ...,     0,     0,     0],
       [    2,  1532,    92, ...,     0,     0,     0],
       ...,
       [    2,   427,   282, ...,     0,     0,     0],
       [    2,  1026, 30356, ...,     0,     0,     0],
       [    2,  6290,  6106, ...,     0,     0,     0]]), array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32), array([[0],
       [0],
       [2],
       [0],
       [2],
       [0],
       [0],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [0],
       [2],
       [2],
       [2],
       [2],
       [0],
       [2],
       [2],
       [1]]))
(array

(Epoch 1) TRAIN LOSS:0.6468 LR:0.00000300:  12%|████▉                                    | 3/25 [00:00<00:03,  5.91it/s]

(array([[    2,  2083,  5259, ...,     0,     0,     0],
       [    2,  6794, 10327, ...,     0,     0,     0],
       [    2,   412, 30371, ...,     0,     0,     0],
       ...,
       [    2,    90,  1214, ...,     0,     0,     0],
       [    2,  1196,   521, ...,     0,     0,     0],
       [    2,   368,  3370, ...,     0,     0,     0]]), array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32), array([[2],
       [2],
       [0],
       [2],
       [0],
       [2],
       [2],
       [2],
       [2],
       [2],
       [0],
       [2],
       [0],
       [0],
       [2],
       [2],
       [0],
       [1],
       [2],
       [0],
       [1],
       [2],
       [2],
       [2],
       [0],
       [2],
       [2],
       [2],
       [1],
       [0],
       [0],
       [0]]))
(array

(Epoch 1) TRAIN LOSS:0.6640 LR:0.00000300:  20%|████████▏                                | 5/25 [00:00<00:03,  6.03it/s]

...

(array([[    2,  5264,  2063, ...,     0,     0,     0],
       [    2,   388, 12456, ...,     0,     0,     0],
       [    2,  3473, 30370, ...,     0,     0,     0],
       ...,
       [    2,  1234, 12600, ...,     0,     0,     0],
       [    2,   515,  1107, ...,     0,     0,     0],
       [    2,   711,  1506, ...,     0,     0,     0]]), array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32), array([[1],
       [2],
       [0],
       [2],
       [2],
       [2],
       [2],
       [2],
       [0],
       [2],
       [1],
       [2],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [2],
       [1],
       [2],
       [0],
       [2],
       [2],
       [2],
       [0],
       [1],
       [2],
       [0],
       [2],
       [2]]))
(array

(Epoch 10) TRAIN LOSS:0.0614 LR:0.00000300: 100%|███████████████████████████████████████| 25/25 [00:05<00:00,  5.27it/s]

(array([[    2,  1617,  3328, ...,     0,     0,     0],
       [    2,  4416,    79, ...,     0,     0,     0],
       [    2,  2382,  2174, ...,     0,     0,     0],
       ...,
       [    2,  1004,    26, ...,     0,     0,     0],
       [    2,    34, 14186, ...,     0,     0,     0],
       [    2,  1054,    34, ...,     0,     0,     0]]), array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32), array([[1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [0],
       [2],
       [2],
       [2],
       [2],
       [0],
       [2],
       [2],
       [1],
       [2],
       [0],
       [0],
       [2],
       [2],
       [2],
       [0],
       [0],
       [2],
       [1],
       [2],
       [2],
       [1],
       [2]]))


(Epoch 10) TRAIN LOSS:0.0614 LR:0.00000300: 100%|███████████████████████████████████████| 25/25 [00:05<00:00,  4.49it/s]

(Epoch 10) TRAIN LOSS:0.0614 ACC:0.99 F1:0.99 REC:0.99 PRE:1.00 LR:0.00000300


In [22]:
# Evaluate on validation
model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm.tqdm(iter(valid_loader), leave=True, total=len(valid_loader))
for i, batch_data in enumerate(pbar):
    batch_seq = batch_data[-1]
    loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
    
    # Calculate total loss
    valid_loss = loss.item()
    total_loss = total_loss + valid_loss

    # Calculate evaluation metrics
    list_hyp += batch_hyp
    list_label += batch_label
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)

    pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
    
metrics = document_sentiment_metrics_fn(list_hyp, list_label)
print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
    total_loss/(i+1), metrics_to_string(metrics)))

VALID LOSS:0.1924 ACC:0.94 F1:0.93 REC:0.93 PRE:0.94: 100%|███████████████████████████████| 7/7 [00:00<00:00, 14.30it/s]

(Epoch 10) VALID LOSS:0.1924 ACC:0.94 F1:0.93 REC:0.93 PRE:0.94
